# Fine-tune FLAN-T5 for CG Classification
In this code, you will see how to fine-tune [google/flan-t5-base](https://huggingface.co/google/flan-t5-base) for CG Classification task using Hugging Face Transformers. Flan t5 is created based on the T5 language model. For the same number of parameters, these models have been fine-tuned on more than 1000 additional tasks covering also more languages.

You will learn how to:

1. [Install requirements](#1-install-requirements)
2. [Load Corpus](#2-load-and-prepare-samsum-dataset)
3. [Fine-tune and evaluate FLAN-T5](#3-fine-tune-and-evaluate-flan-t5)
4. [Run Inference](#4-run-inference)

## FLAN-T5
FLAN-T5 released with the [Scaling Instruction-Finetuned Language Models](https://arxiv.org/pdf/2210.11416.pdf) paper is an enhanced version of T5 that has been finetuned in a mixture of tasks. The paper explores instruction finetuning with a particular focus on (1) scaling the number of tasks, (2) scaling the model size, and (3) finetuning on chain-of-thought data. The paper discovers that overall instruction finetuning is a general method for improving the performance and usability of pretrained language models.

## 1. Install requirements

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [ ]:
# python
!pip install pytesseract transformers==4.28.1 datasets evaluate rouge-score nltk tensorboard py7zr

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

## Connect to Drive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Load and prepare dataset

In [ ]:
import pickle

f = open("/content/drive/MyDrive/Corpus/CG_Corpus/cg_3to1_2previous_event_selection.dat", "rb")
dataset = pickle.load(f)
f.close()

In [ ]:
SUM = 0
for record in dataset['test']:
  if record['Bel(A)'] == 3:
    SUM+=1
print(SUM)

20


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 970
    })
    test: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 325
    })
})

In [ ]:
import pandas as pd
from datasets import Dataset

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
train_df['CG(A)'] = train_df['CG(A)'].astype(str)
test_df['CG(A)'] = test_df['CG(A)'].astype(str)
train_df['CG(B)'] = train_df['CG(B)'].astype(str)
test_df['CG(B)'] = test_df['CG(B)'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)

dataset['train'] = dataset['train'].shuffle()

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 970
    })
    test: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 325
    })
})

In [ ]:
def cal_class_support(class_number, CG_col, corpora):
  SUM = 0
  for record in dataset[corpora]:
    if record[CG_col] == class_number: SUM+=1
  return SUM

CG_A_Train_Support = [cal_class_support(1, "CG(A)", "train"), cal_class_support(2, "CG(A)", "train"), cal_class_support(3, "CG(A)", "train"), cal_class_support(4, "CG(A)", "train"), cal_class_support(0, "CG(A)", "train")]
CG_A_Test_Support = [cal_class_support(1, "CG(A)", "test"), cal_class_support(2, "CG(A)", "test"), cal_class_support(3, "CG(A)", "test"), cal_class_support(4, "CG(A)", "test"), cal_class_support(0, "CG(A)", "test")]
CG_B_Train_Support = [cal_class_support(1, "CG(B)", "train"), cal_class_support(2, "CG(B)", "train"), cal_class_support(3, "CG(B)", "train"), cal_class_support(4, "CG(B)", "train"), cal_class_support(0, "CG(B)", "train")]
CG_B_Test_Support = [cal_class_support(1, "CG(B)", "test"), cal_class_support(2, "CG(B)", "test"), cal_class_support(3, "CG(B)", "test"), cal_class_support(4, "CG(B)", "test"), cal_class_support(0, "CG(B)", "test")]

print(f"CG(A) Train : {CG_A_Train_Support}\nCG(A) Test  : {CG_A_Test_Support}\nCG(B) Train : {CG_B_Train_Support}\nCG(B) Test  : {CG_B_Test_Support}")

CG(A) Train : [0, 0, 0, 0, 0]
CG(A) Test  : [0, 0, 0, 0, 0]
CG(B) Train : [0, 0, 0, 0, 0]
CG(B) Test  : [0, 0, 0, 0, 0]


In [ ]:
train_df = dataset['train'].to_pandas()
train_df = train_df[train_df['CG(A)'] != '0']
dataset['train'] = Dataset.from_pandas(train_df)
dataset['train'] = dataset['train'].remove_columns('__index_level_0__')

test_df = dataset['test'].to_pandas()
test_df = test_df[test_df['CG(A)'] != '0']
dataset['test'] = Dataset.from_pandas(test_df)
dataset['test'] = dataset['test'].remove_columns('__index_level_0__')

In [ ]:
dataset['train'][20]

{'Speaker': 'A',
 'Sentence_Number': 33,
 'Sentence': 'A: %um, and the doctor showed up around three and &Mae was born at five twenty five.  ',
 'Event': "Previous Sentences: Whether the hospital staff can hold one back with the labor or not depends on how much medicine the hospital staff gives one The hospital staff gives one medicine during labor A thinks the hospital staff were holding Mae's birth mom back a little bit because the hospital staff wanted the doctor in Mae's birth mom delivery room The hospital staff were holding Mae's birth mom back a little bit because the hospital staff wanted the doctor in Mae's birth mom delivery room The hospital staff were holding Mae's birth mom back a little bit The hospital staff wanted the doctor in Mae's birth mom delivery room \nTarget Sentence: The doctor showed up around three",
 'Target_Event': 'The doctor showed up around three',
 'Bel(A)': 1,
 'Bel(B)': 1,
 'CG(A)': '1',
 'CG(B)': '1'}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 889
    })
    test: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 311
    })
})

Lets checkout an example of the dataset.

In [ ]:
from random import randrange

sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"Speaker is <{sample['Speaker']}>")
print(f"Text: \n{sample['Event']}\n---------------")
print(f"CG(A): \n{sample['CG(A)']}\n---------------")

Speaker is <A>
Text: 
Previous Sentences: Mae's birth mom was supposed to go into labor at six thirty in the morning but Mae's birth mom called and the hospital didn't have any beds Mae's birth mom was supposed to go into labor at six thirty in the morning Mae's birth mom went into labot at six thirty in the morning Mae's birth mom called The hospital didn't have any beds So Mae's mom called A and Robert So Mae's mom told A and Robert about events im 8.2, 8.4 and 8.5 The hospital did get a bed eventually Mae's mom went into labor at eight Mae's mom went into labor at seven A guesses Mae's birth mom went into labor at seven thirty Mae's birth mom went into labor at seven thirty The hospital staff started Mae's birth mom at eight so A and Robert hopped in the car and drove down to be at the hospital for the day The hospital staff started Mae's birth mom at eight A and Robert hopped in the car 
Target Sentence: A and Robert drove down to be at the hospital for the day
---------------
CG(A

Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

text2text-generation task: our model will take a utterance as input and generate events as output. For this we want to understand how long our input and output will be to be able to efficiently batch our data.

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Event"], truncation=True), batched=True, remove_columns=['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["CG(A)"], truncation=True), batched=True, remove_columns=['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Max source length: 417


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Max target length: 2


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    # inputs = ["Bel(A): " + item for item in sample["Event"]]
    inputs = [item for item in sample["Event"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["CG(A)"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5


In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

Trainer: We are going to use evaluate library to evaluate the rogue score.

We are going to use `evaluate` library to evaluate the `rogue` score.

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("f1")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training. We are leveraging the [Hugging Face Hub](https://huggingface.co/models) integration of the `Trainer` to automatically push our checkpoints, logs and metrics during training into a repository.

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-event-extraction"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=12,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=300,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    # push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

We can start our training by using the `train` method of the `Trainer`.

In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Gen Len
1,0.380200,0.342124,29.376500,2.000000
2,0.226300,0.296764,44.144300,2.000000
3,0.163700,0.366322,44.368900,2.000000
4,0.104500,0.534907,43.740400,2.000000
5,0.061400,1.171835,44.133100,2.000000
6,0.042800,0.899144,53.674300,2.000000
7,0.037000,1.066207,54.746100,2.000000
8,0.023300,1.282858,48.897400,2.000000
9,0.008800,1.362348,52.532500,2.000000
10,0.003900,1.652406,50.538400,2.000000


TrainOutput(global_step=1344, training_loss=0.08781368595581236, metrics={'train_runtime': 1571.7447, 'train_samples_per_second': 6.787, 'train_steps_per_second': 0.855, 'total_flos': 6049446032867328.0, 'train_loss': 0.08781368595581236, 'epoch': 12.0})

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [ ]:
trainer.evaluate()

{'eval_loss': 1.6234279870986938,
 'eval_f1': 50.8514,
 'eval_gen_len': 2.0,
 'eval_runtime': 22.9934,
 'eval_samples_per_second': 13.526,
 'eval_steps_per_second': 1.696,
 'epoch': 12.0}

## 4. Run Inference and Classification Report


In [ ]:
results_dict = {
    'Speaker': [],
    'Sentence_Number': [],
    'Sentence': [],
    'Event': [],
    'Target_Event': [],
    'Bel(A)': [],
    'Bel(B)': [],
    'Predicted CG(A)': [],
    'Predicted CG(B)': [],
    'CG(A)': [],
    'CG(B)': [],
}

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  # text = "Bel(A): " + dataset['test']['Event'][i]
  text = dataset['test']['Event'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['CG(A)'][i])

  results_dict['Speaker'].append(dataset['test']['Speaker'][i])
  results_dict['Sentence_Number'].append(dataset['test']['Sentence_Number'][i])
  results_dict['Sentence'].append(dataset['test']['Sentence'][i])
  results_dict['Event'].append(dataset['test']['Event'][i])
  results_dict['Target_Event'].append(dataset['test']['Target_Event'][i])
  results_dict['Bel(A)'].append(dataset['test']['Bel(A)'][i])
  results_dict['Bel(B)'].append(dataset['test']['Bel(B)'][i])
  results_dict['Predicted CG(A)'].append(prediction)
  results_dict['Predicted CG(B)'].append(prediction)
  results_dict['CG(A)'].append(dataset['test']['CG(A)'][i])
  results_dict['CG(B)'].append(dataset['test']['CG(B)'][i])

  progress_bar.update(1)

  0%|          | 0/311 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           1       0.83      0.92      0.88       245
           2       0.64      0.47      0.54        30
           3       0.17      0.08      0.11        36

    accuracy                           0.78       311
   macro avg       0.55      0.49      0.51       311
weighted avg       0.74      0.78      0.75       311



In [ ]:
results_df = pd.DataFrame.from_dict(results_dict)

In [ ]:
results_df

,Speaker,Sentence_Number,Sentence,Event,Target_Event,Bel(A),Bel(B),Predicted CG(A),Predicted CG(B),CG(A),CG(B)
0,B,1,B: %um I took them to %uh &Jill’s and they spe...,Previous Sentences: \nTarget Sentence: B took ...,B took the kids to Jill's,1,1,1,1,1,1
1,B,1,None,Previous Sentences: B took the kids to Jill's ...,The kids spent two days at Jill's,1,1,1,1,1,1
2,B,1,None,Previous Sentences: B took the kids to Jill's ...,B guesses Jill couldn't take the kids,1,1,1,1,1,1
3,B,1,None,Previous Sentences: B took the kids to Jill's ...,Jill couldn't take the kids,3,3,1,1,1,1
4,B,1,None,Previous Sentences: B took the kids to Jill's ...,The kid's mom and dad came,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
306,A,144,None,Previous Sentences: A asks B if B knows what A...,A's sons never treat one another like A and B'...,3,3,1,1,1,1
307,B,145,B: True. I don’t think my kids will be that wa...,Previous Sentences: B doesn't know what A said...,B doesn't think B's kids will be like A and B'...,1,1,1,1,1,1
308,B,145,None,Previous Sentences: B doesn't know what A said...,B's kids will be like A and B's mom and dad,3,3,1,1,1,1
309,A,146,A: And he just looked at me. [channel noise],Previous Sentences: A said A hopes that A's so...,A and B's dad just looked at A,1,1,1,1,1,1


In [ ]:
results_df.to_csv('results.csv')

### Using Bel(A) trained model for test on Bel(B)

we should remove Bel(B)=0 for calculate correct classification reports

In [ ]:
test_df = dataset['test'].to_pandas()
test_df = test_df[test_df['CG(B)'] != '0']
dataset['test'] = Dataset.from_pandas(test_df)
dataset['test'] = dataset['test'].remove_columns('__index_level_0__')

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['Event'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['CG(B)'][i])

  progress_bar.update(1)

  0%|          | 0/310 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           1       0.83      0.92      0.88       245
           2       0.62      0.46      0.53        28
           3       0.22      0.11      0.15        37

    accuracy                           0.78       310
   macro avg       0.56      0.50      0.52       310
weighted avg       0.74      0.78      0.76       310



### Show

In [ ]:
for i in range(len(predictions_list)):
  if predictions_list[i] == labels_list[i]: print(predictions_list[i], labels_list[i])
  else: print(predictions_list[i], labels_list[i], "Incorrect")


1 1
1 1
1 1
4 4
1 1
4 1 Incorrect
1 1
1 1
1 1
3 1 Incorrect
1 1
4 3 Incorrect
1 1
2 1 Incorrect
1 1
1 1
1 1
3 1 Incorrect
1 1
1 1
1 1
3 1 Incorrect
3 1 Incorrect
1 1
1 1
1 3 Incorrect
2 2
1 1
1 1
1 1
4 4
1 1
1 1
1 1
1 1
1 1
1 1
3 3
1 1
1 1
1 1
1 1
1 1
1 1
1 1
4 4
2 1 Incorrect
1 1
1 1
1 1
1 1
1 1
2 3 Incorrect
4 1 Incorrect
3 1 Incorrect
3 3
1 1
3 3
1 1
1 1
3 4 Incorrect
4 4
1 1
1 1
1 1
2 1 Incorrect
1 1
1 1
1 1
1 1
1 1
1 1
1 2 Incorrect
1 1
1 1
1 1
1 1
1 1
4 4
4 1 Incorrect
1 1
1 1
1 1
4 1 Incorrect
1 1
1 1
4 1 Incorrect
2 2
2 2
1 1
1 1
1 4 Incorrect
1 1
1 1
3 2 Incorrect
4 3 Incorrect
1 1
1 1
2 2
1 2 Incorrect
1 1
3 1 Incorrect
1 1
1 1
1 1
1 1
1 1
1 4 Incorrect
1 2 Incorrect
1 1
1 3 Incorrect
1 1
3 2 Incorrect
1 1
1 1
1 1
1 1
2 1 Incorrect
1 1
1 1
1 1
3 3
1 1
1 1
2 1 Incorrect
1 1
4 4
1 1
1 1
1 1
4 1 Incorrect
1 1
1 1
1 1
1 1
1 1
1 1
1 1
3 2 Incorrect
3 1 Incorrect
3 1 Incorrect
1 1
4 1 Incorrect
2 2
1 1
1 1
4 1 Incorrect
1 1
1 1
1 1
4 1 Incorrect
1 4 Incorrect
1 1
1 1
4 1 Incorrect


### Save Model

In [ ]:
save_directory = "/content/drive/MyDrive/Common Ground Docs/Models/FlanT5_Final_Model_Bel_Classification_3to1_49BelA_37BelB"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

### load Model and Test pretrained model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [ ]:
model.to('cuda')

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['Event'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['Bel(B)'][i])

  progress_bar.update(1)

  0%|          | 0/242 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           1       0.93      0.86      0.90       200
           2       0.46      0.43      0.44        14
           3       0.35      0.47      0.40        17
           4       0.30      0.55      0.39        11

    accuracy                           0.80       242
   macro avg       0.51      0.58      0.53       242
weighted avg       0.83      0.80      0.81       242

